# Clustering Crypto

In [74]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [75]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
crypto_data = pd.read_csv(file_path)
crypto_data.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [76]:
crypto_trading_df = crypto_data[crypto_data['IsTrading'] == True]
crypto_trading_df.set_index('Unnamed: 0', inplace=True)
crypto_trading_df.index.name = None
crypto_trading_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [77]:
# Keep all the cryptocurrencies that are being traded.
crypto_data_trading_df = crypto_data[crypto_data["IsTrading"] == True]
crypto_data_trading_df.set_index('Unnamed: 0', inplace = True)
crypto_data_trading_df.index.name = None
crypto_data_trading_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [78]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_data_trading_df.isnull().sum()
#crypto_data_trading_df.count()

CoinName             0
Algorithm            0
IsTrading            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [79]:
#Double checking that the above is correct
crypto_data_trading_workingalg_df = crypto_data_trading_df[crypto_data_trading_df["Algorithm"].notnull()]
crypto_data_trading_workingalg_df.head()
crypto_data_trading_workingalg_df.count()

CoinName           1144
Algorithm          1144
IsTrading          1144
ProofType          1144
TotalCoinsMined     685
TotalCoinSupply    1144
dtype: int64

In [80]:
# Remove the "IsTrading" column. 
crypto_remove_IsTrading_df = crypto_data_trading_workingalg_df.drop(columns="IsTrading", axis=3)
crypto_remove_IsTrading_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [81]:
# Remove rows that have at least 1 null value.
crypto_cleaned_df = crypto_remove_IsTrading_df.dropna()
crypto_cleaned_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [82]:
crypto_cleaned_df.count()

CoinName           685
Algorithm          685
ProofType          685
TotalCoinsMined    685
TotalCoinSupply    685
dtype: int64

In [83]:
# Keep the rows where coins are mined.
crypto_mined_df = crypto_cleaned_df[crypto_cleaned_df["TotalCoinsMined"] >0]
crypto_mined_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [84]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name_df = crypto_mined_df[["CoinName"]]
crypto_name_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [85]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_mined_no_CoinName_df = crypto_mined_df.drop(columns=["CoinName"], axis = 2)
crypto_mined_no_CoinName_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [86]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_mined_no_CoinName_df, columns=["Algorithm", "ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
# Standardize the data with StandardScaler().
crypto_df = StandardScaler().fit_transform(X)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [88]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
pca_crypto = pca.fit_transform(crypto_df)
pca_crypto

array([[-0.32802171,  1.01265751, -0.59219909],
       [-0.31146743,  1.01285041, -0.59259206],
       [ 2.29838127,  1.67329272, -0.632749  ],
       ...,
       [ 0.3076098 , -2.29900582,  0.37973177],
       [-0.14176814, -2.04956499,  0.35648885],
       [-0.30736187,  0.80096193, -0.2609617 ]])

In [89]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data = pca_crypto, columns = ["PC 1", "PC 2", "PC 3"], index = crypto_mined_no_CoinName_df.index)
pcs_df

,PC 1,PC 2,PC 3
42,-0.328022,1.012658,-0.592199
404,-0.311467,1.012850,-0.592592
1337,2.298381,1.673293,-0.632749
BTC,-0.151660,-1.276032,0.179758
ETH,-0.163700,-2.017047,0.358615
...,...,...,...
ZEPH,2.558813,0.774570,-0.050523
GAP,-0.326084,1.012550,-0.592227
BDX,0.307610,-2.299006,0.379732
ZEN,-0.141768,-2.049565,0.356489


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [90]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
elbow_data = {"k":k, "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [91]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)

[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 0 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 0 3 3 0 0 3 0 0 3 3 0 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 0 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 0 0
 3 3 0 3 3 3 3 0 1 1 3 3 3 0 1 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 1 3 0 3 3 0 0 3 0 3 3 0 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

In [92]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.DataFrame({
    "Algorithm":crypto_mined_no_CoinName_df["Algorithm"],
    "ProofType":crypto_mined_no_CoinName_df["ProofType"],
    "TotalCoinsMined":crypto_mined_no_CoinName_df["TotalCoinsMined"],
    "TotalCoinSupply":crypto_mined_no_CoinName_df["TotalCoinSupply"],
    "PC1":pcs_df["PC 1"],
    "PC2":pcs_df["PC 2"],
    "PC3":pcs_df["PC 3"],
    "CoinName": crypto_name_df["CoinName"],
    "Class": predictions
    
})

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.328022,1.012658,-0.592199,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.311467,1.012850,-0.592592,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.298381,1.673293,-0.632749,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.151660,-1.276032,0.179758,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.163700,-2.017047,0.358615,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.172464,-1.139671,-0.018828,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.364802,1.216618,-0.514919,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.156610,-2.224379,0.451944,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.162155,-2.017140,0.358594,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.141767,-2.049565,0.356489,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [98]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    hover_data=["Algorithm"],
    hover_name="CoinName",
    color="Class",
    symbol="Class",
    width=800
)

fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [99]:
# Create a table with tradable cryptocurrencies.
columns = ["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"]
clustered_df.hvplot.table(columns)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [103]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {clustered_df.CoinName.size} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [107]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_scaled_df = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
clustered_scaled_df

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [109]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
new_df = pd.DataFrame(clustered_scaled_df, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)


# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
new_df = pd.DataFrame({
    "TotalCoinSupply": new_df["TotalCoinSupply"],
    "TotalCoinsMined": new_df["TotalCoinsMined"],
    "CoinName": clustered_df["CoinName"],
    "Class": clustered_df["Class"]
})

new_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,3
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3
ZEC,2.100000e-05,0.000007,ZCash,3


In [115]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
new_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', by='Class', 
                  legend='right', hover_cols=["CoinName"])


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)